### Installing libraries if need them...

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
//import $ivy.`sh.almond::almond-spark:0.3
import $ivy.`sh.almond::ammonite-spark:0.4.0`
import $ivy.`org.datasyslab:geospark:1.2.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                   // Or use any other 2.x version here
//import $ivy.`sh.almond::almond-spark:0.3

import $ivy.$                                

import $ivy.$                              

import org.apache.log4j.{Level, Logger}


### Importing classes...

In [2]:
import org.apache.spark.serializer.KryoSerializer
import org.apache.spark.storage.StorageLevel
import org.datasyslab.geospark.enums.{GridType, IndexType}
import org.datasyslab.geospark.spatialOperator.JoinQuery
import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader
import scala.collection.JavaConverters._
import java.io._

import org.apache.spark.serializer.KryoSerializer

import org.apache.spark.storage.StorageLevel

import org.datasyslab.geospark.enums.{GridType, IndexType}

import org.datasyslab.geospark.spatialOperator.JoinQuery

import org.datasyslab.geospark.formatMapper.shapefileParser.ShapefileReader

import scala.collection.JavaConverters._

import java.io._

### Starting session...

In [3]:
import org.apache.spark.sql._

val spark = AmmoniteSparkSession.builder()
    .config("spark.serializer",classOf[KryoSerializer].getName)
    .master("local[*]").appName("Areal")
    .getOrCreate()
import spark.implicits._
val appID = spark.sparkContext.applicationId

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@43a9e0f7
import spark.implicits._

appID: String = "local-1557272476150"

### Reading source...

In [4]:
var source = "/home/acald013/Datasets/gdf_2000"
val sourceRDD = ShapefileReader.readToGeometryRDD(spark.sparkContext, source)
val sourceIDs = sourceRDD.rawSpatialRDD.rdd.zipWithUniqueId.map{ s =>
    val id = s._2
    val geom = s._1
    geom.setUserData(s"${id}\t${geom.getUserData.toString()}")
    geom
}.persist(StorageLevel.MEMORY_ONLY)
sourceRDD.setRawSpatialRDD(sourceIDs)
val nSourceRDD = sourceRDD.rawSpatialRDD.rdd.count()

source: String = "/home/acald013/Datasets/gdf_2000"
sourceRDD: org.datasyslab.geospark.spatialRDD.SpatialRDD[com.vividsolutions.jts.geom.Geometry] = org.datasyslab.geospark.spatialRDD.SpatialRDD@543ad71
sourceIDs: org.apache.spark.rdd.RDD[com.vividsolutions.jts.geom.Geometry] = MapPartitionsRDD[5] at map at cmd3.sc:3
nSourceRDD: Long = 72693L

### Reading target...

In [5]:
var target = "/home/acald013/Datasets/gdf_1990"
val targetRDD = ShapefileReader.readToGeometryRDD(spark.sparkContext, target)
val targetIDs = targetRDD.rawSpatialRDD.rdd.zipWithUniqueId.map{ t =>
    val id = t._2
    val geom = t._1
    geom.setUserData(s"${id}\t${geom.getUserData.toString()}")
    geom
}.persist(StorageLevel.MEMORY_ONLY)
targetRDD.setRawSpatialRDD(targetIDs)
val nTargetRDD = targetRDD.rawSpatialRDD.rdd.count()

target: String = "/home/acald013/Datasets/gdf_1990"
targetRDD: org.datasyslab.geospark.spatialRDD.SpatialRDD[com.vividsolutions.jts.geom.Geometry] = org.datasyslab.geospark.spatialRDD.SpatialRDD@19272eb3
targetIDs: org.apache.spark.rdd.RDD[com.vividsolutions.jts.geom.Geometry] = MapPartitionsRDD[11] at map at cmd4.sc:3
nTargetRDD: Long = 61332L

### Performing spatial join...

In [ ]:
sourceRDD.rawSpatialRDD.rdd.map{s => 
    val arr = s.getUserData().toString().split("\t")
    (arr(12), arr(14), s.toText())
}.toDF()
//.filter(_.getString(0) == "01001020300")
.show(truncate = true)

In [9]:
targetRDD.rawSpatialRDD.rdd.map(s => s.getUserData().toString()).take(15).foreach(println)

0	01023009567		0103000000010000007B070000DC9C4A06801A56C08C852172FA164040C9C9C4AD821A56C02F87DD770C174040E6948098841A56C0D36BB3B112174040FED30D14781A56C0F209D9791B174040745FCE6C571A56C0DDEBA4BE2C1740407024D060531A56C053B4722F301740409696917A4F1A56C0868DB27E33174040C32D
1	01023009568		010300000001000000F6090000A3E5400FB51456C024624A24D1034040F0517FBDC21456C024624A24D1034040BA30D28BDA1456C033E02C25CB0340402BC1E270E61456C02A029CDEC5034040195932C7F21456C08FDE701FB9034040CFF6E80DF71456C043A9BD88B6034040925CFE43FA1456C03E20D099B4034040925C
2	01023009569		0103000000010000008B040000840EBA84431D56C001C11C3D7ECF3F40A699EE75521D56C0707B82C476CF3F40FD6B79E57A1D56C01841632651CF3F400AB952CF821D56C0F9BF232A54CF3F4009371955861D56C0EE04FBAF73CF3F408D9AAF928F1D56C0AF9811DE1ED03F408E058541991D56C01363997E89D03F401DC9
3	01023009570		01030000000100000082020000FA7C9411170C56C00E881057CEB23F4023DBF97E6A0C56C0C6DB4AAFCDB23F40087767EDB60C56C07F2F8507CDB23F4062C092AB580D56C050DF32A7CBB23F40632AFD84B30D56C044FD2

In [6]:
val considerBoundaryIntersection = true // Only return gemeotries fully covered by each query window in queryWindowRDD
val buildOnSpatialPartitionedRDD = true // Set to TRUE only if run join query
val usingIndex = true
val partitions = 1024

sourceRDD.analyze()
sourceRDD.spatialPartitioning(GridType.QUADTREE, partitions)
targetRDD.spatialPartitioning(sourceRDD.getPartitioner)
sourceRDD.buildIndex(IndexType.QUADTREE, buildOnSpatialPartitionedRDD)

val joined = JoinQuery.SpatialJoinQuery(sourceRDD, targetRDD, usingIndex, considerBoundaryIntersection)
val nJoined = joined.count()

considerBoundaryIntersection: Boolean = true
buildOnSpatialPartitionedRDD: Boolean = true
usingIndex: Boolean = true
partitions: Int = 1024
res5_4: Boolean = true
joined: org.apache.spark.api.java.JavaPairRDD[com.vividsolutions.jts.geom.Geometry, java.util.HashSet[com.vividsolutions.jts.geom.Geometry]] = org.apache.spark.api.java.JavaPairRDD@7ad91aa1
nJoined: Long = 61313L

### Postprocessing the join results...

In [7]:
val flattened = joined.rdd.flatMap{ pair =>
    val a = pair._1
    pair._2.asScala.map(b => (a, b))
}
val nFlattened = flattened.count()

flattened: org.apache.spark.rdd.RDD[(com.vividsolutions.jts.geom.Geometry, com.vividsolutions.jts.geom.Geometry)] = MapPartitionsRDD[24] at flatMap at cmd6.sc:1
nFlattened: Long = 357427L

### Computing the area...

In [8]:
val areal = flattened.map{ pair =>
    val source_id  = pair._1.getUserData.toString().split("\t")(0)
    val target_id  = pair._2.getUserData.toString().split("\t")(0)
    val area = pair._1.intersection(pair._2).getArea
    (source_id, target_id, area)
}
val nAreal = areal.count()

areal: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[25] at map at cmd7.sc:1
nAreal: Long = 357427L

### Reporting the results...

In [9]:
areal.toDF("SourceID", "TargetID", "Area").show(truncate=false)

+--------+--------+---------------------+
|SourceID|TargetID|Area                 |
+--------+--------+---------------------+
|22193   |28801   |1.6695735321167273E-6|
|22193   |28841   |5.571185507966348E-7 |
|22193   |28888   |3.7806561723031056E-5|
|22193   |28906   |1.1618461940393888E-6|
|25527   |31078   |2.5008364368929475E-6|
|25527   |31080   |0.002344273245773022 |
|25527   |31081   |1.286372551098695E-5 |
|25527   |31079   |8.344323279225162E-6 |
|37570   |43943   |6.932227441694678E-5 |
|37570   |43933   |0.1662606571830884   |
|37570   |43953   |8.687769861680146E-7 |
|37570   |44004   |7.392856252058025E-5 |
|37570   |44006   |1.0873067647508585E-4|
|37570   |43937   |1.158960074807664E-4 |
|37570   |47552   |4.338532850032905E-6 |
|37570   |43939   |3.5375703881186553E-6|
|37570   |43940   |1.0177161635319456E-5|
|37570   |43934   |6.810584669886284E-5 |
|37570   |47559   |1.159701892252949E-8 |
|13873   |18515   |9.74852410303433E-5  |
+--------+--------+---------------

### Closing the session...

In [10]:
spark.close()